In [1]:
import cv2
from ultralytics import YOLO
import os

# Load YOLOv8 models for vehicle detection and number plate detection
vehicle_model = YOLO('yolov8n.pt')  # Replace with your vehicle detection model
plate_model = YOLO('number_plate_model.pt')  # Replace with your number plate detection model

# Path to your video file
video_path = 'input/output_video.mp4'

# Directory to save extracted number plates
output_dir = 'extracted_number_plates'
os.makedirs(output_dir, exist_ok=True)

    

In [ ]:
# Open the video file
cap = cv2.VideoCapture(video_path)

# Check if the video file opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

frame_count = 0
plate_count = 0
frame_batch_size = 10  # Pick only one frame pet batch.

# Process each frame of the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Increment frame count
    frame_count += 1

    # Skip frames
    if frame_count % (frame_batch_size) != 0:
        continue

    # Perform vehicle detection on the frame
    vehicle_results = vehicle_model(frame)

    # Ensure results is not empty and contains the boxes attribute
    if vehicle_results and hasattr(vehicle_results[0], 'boxes'):
        # Iterate over detected vehicles
        for vehicle in vehicle_results[0].boxes:
            # Extract bounding box coordinates for vehicle
            x1, y1, x2, y2 = map(int, vehicle.xyxy[0])

            # Crop the detected vehicle region
            vehicle_region = frame[y1:y2, x1:x2]

            # Perform number plate detection on the vehicle region
            plate_results = plate_model(vehicle_region)

            if plate_results and hasattr(plate_results[0], 'boxes'):
                for plate in plate_results[0].boxes:
                    # Extract bounding box coordinates for number plate
                    px1, py1, px2, py2 = map(int, plate.xyxy[0])

                    # Adjust the coordinates relative to the original frame
                    px1 += x1
                    py1 += y1
                    px2 += x1
                    py2 += y1

                    # Crop the detected number plate region
                    number_plate = frame[py1:py2, px1:px2]

                    # Save the cropped number plate image
                    plate_file_path = os.path.join(output_dir, f'plate_{plate_count}.png')
                    cv2.imwrite(plate_file_path, number_plate)
                    plate_count += 1

                    print(f'Saved number plate: {plate_file_path}')

    # Display the frame with detections (optional)
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all OpenCV windows
cap.release()
# cv2.destroyAllWindows()  # Uncomment if using cv2.imshow

print(f'Total number plates extracted: {plate_count}')
